# Combined Indicators

This notebook is meant to compile all of the indicators into a single dataframe for easy analysis.  

The steps required to do this are:  
* Gather data from source web sites  
* Clean data and choose important data points to put into a single .csv or database  
* Eliminate trends from data using Mayer's multiple and/or z-scores  

Table of Contents:  
* Quarterly GDP Data  
* Monthly Chicago Fed National Activity Index  

### Quarterly GDP Data

This data requires use of EMAs and Mayer's multiple to eliminate trend effects.

In [108]:
import pandas as pd

df_gdp = pd.read_csv('../0_inputs/economic_indicators/bea_116_real_gdp_cleaned.csv')
df_gdp.tail(3)

,year,quarter,gross_domestic_product,personal_consumption_expenditures,goods,durable_goods,nondurable_goods,services,gross_private_domestic_investment,fixed_investment,...,services.1,imports,goods1,services1,government_consumption_expenditures_and_gross_investment,federal,national_defense,nondefense,state_and_local,residual
298,2021,Q3,19478.9,13732.4,5518.3,2158.5,3394,8378.5,3609.7,3585,...,599.8,3589.6,3122.8,492.2,3381.6,1339.1,794.3,543.9,2042.1,-57.1
299,2021,Q4,19806.3,13818.4,5533.9,2171.9,3397.1,8446.2,3903.0,3609.1,...,627.7,3740.8,3261.1,507.6,3359.0,1324.3,782,541.2,2034,-44.4
300,2022,Q1,19735.9,13911.5,5532.3,2194,3375.9,8534.9,3925.3,3673.2,...,633.6,3896.3,3416.7,512.7,3335.8,1304.3,764.8,538.2,2030.1,-19.5


In [109]:
# This data contains --- where there are blanks
df_gdp = df_gdp.replace(to_replace='---',value=0)

In [ ]:
# Interquartile Range Code to eliminate outliers
# quartiles = df_gdp2['_mayer'].quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq
# iqr_1_5 = iqr * 1.5
# lower_bound = lowerq - iqr_1_5
# upper_bound = upperq + iqr_1_5

In [110]:
cols = list(df_gdp.columns)
col_zscore_list = ['year','quarter']
col_zscore_chg_list = []

for col in cols:
    col_5yr = col + '_5yr'
    col_mayer = col + '_mayer'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        df_gdp[col_5yr] = df_gdp[col].ewm(span=20,adjust=False).mean()
        df_gdp[col_mayer] = (df_gdp[col] - df_gdp[col_5yr]) / df_gdp[col_5yr] * 100
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')

# Eliminate first 20 rows of data
df_gdp2 = df_gdp[20:]

cols = list(df_gdp2.columns)

for col in cols:
    col_mayer = col + '_mayer'
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        df_gdp2[col_zscore] = (df_gdp2[col_mayer] - df_gdp2[col_mayer].mean())/df_gdp2[col_mayer].std(ddof=0)
        df_gdp2[col_chg] = df_gdp2[col_mayer].diff()
        df_gdp2[col_chg_zscore] = (df_gdp2[col_chg] - df_gdp2[col_chg].mean())/df_gdp2[col_chg].std(ddof=0)
    except:
        print(f'No numeric types in {col}.')
        
df2 = df_gdp2[col_zscore_list+col_zscore_chg_list]
df2.drop(['year_zscore','year_chg_zscore'],axis=1,inplace=True)
df2.to_csv('./Output/Quarterly_Data/GDP_116.csv')
df2.tail(5)

No numeric types in quarter.
No numeric types in goods.
No numeric types in durable_goods.
No numeric types in nondurable_goods.
No numeric types in services.
No numeric types in fixed_investment.
No numeric types in nonresidential.
No numeric types in structures.
No numeric types in equipment.
No numeric types in intellectual_property_products.
No numeric types in residential.
No numeric types in change_in_private_inventories.
No numeric types in net_exports_of_goods_and_services.
No numeric types in goods.1.
No numeric types in services.1.
No numeric types in goods1.
No numeric types in services1.
No numeric types in federal.
No numeric types in national_defense.
No numeric types in nondefense.
No numeric types in state_and_local.
No numeric types in quarter.
No numeric types in goods.
No numeric types in durable_goods.
No numeric types in nondurable_goods.
No numeric types in services.
No numeric types in fixed_investment.
No numeric types in nonresidential.
No numeric types in stru

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,year,quarter,gross_domestic_product_zscore,personal_consumption_expenditures_zscore,gross_private_domestic_investment_zscore,exports_zscore,imports_zscore,government_consumption_expenditures_and_gross_investment_zscore,residual_zscore,gross_domestic_product_chg_zscore,personal_consumption_expenditures_chg_zscore,gross_private_domestic_investment_chg_zscore,exports_chg_zscore,imports_chg_zscore,government_consumption_expenditures_and_gross_investment_chg_zscore,residual_chg_zscore
296,2021,Q1,-1.184664,-0.942987,-0.202502,-2.043672,-0.819065,-0.264459,0.173402,1.079342,2.205351,-0.356430,-0.056140,0.436092,0.703876,-0.043966
297,2021,Q2,-0.764429,-0.074523,-0.404416,-1.785642,-0.689532,-0.410876,0.164791,1.075490,2.150175,-0.415151,0.530520,0.261518,-0.635334,-0.005732
298,2021,Q3,-0.751567,-0.183255,-0.131302,-1.908862,-0.647682,-0.424347,0.043970,0.052039,-0.275819,0.540237,-0.242068,0.084238,0.052300,-0.086694
299,2021,Q4,-0.356906,-0.228820,0.713943,-1.291313,-0.210186,-0.580266,0.011893,1.011239,-0.119000,1.690921,1.259073,0.884166,-0.684481,-0.022663
300,2022,Q1,-0.708174,-0.251808,0.593677,-1.484277,0.183875,-0.723472,-0.049509,-0.862770,-0.062949,-0.250938,-0.383403,0.796347,-0.618728,-0.043822


In [91]:
# Code used to test accuracy of ema calculation
# import yfinance as yf

# ticker = yf.Ticker('SPY')
# # Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# ticker_hist = ticker.history(period="5y")
# ticker_hist2 = ticker_hist.reset_index()
# df = ticker_hist2

# df['ema_20'] = df['Close'].ewm(span=20,adjust=False).mean()
# df[['ema_20']]

In [92]:
# df_T['EMA'] = df_T.iloc[:,0].ewm(span=40,adjust=False).mean()

### Chicago Fed National Activity Index (CFNAI)

In [6]:
import pandas as pd

df_cfnai = pd.read_csv('../0_inputs/economic_indicators/cfnai-data-series-xlsx_cleaned.csv')

# Eliminate first 2 rows for 3 month MA data
df_cfnai = df_cfnai[2:]
df_cfnai.tail(3)

,year,month,prod_and_income,employment,consumption_and_housing,sales_orders_inventories,cfnai,cfnai_ma3,diffusion
658,2022,1,0.17,0.13,0.28,0.16,0.75,0.44,0.33
659,2022,2,0.30,0.31,-0.04,-0.03,0.54,0.43,0.31
660,2022,3,0.27,0.16,-0.04,0.06,0.44,0.57,0.45


In [7]:
cols = list(df_cfnai.columns)
col_zscore_list = ['year','month']
col_zscore_chg_list = []

for col in cols:
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
    try:
        df_cfnai[col_zscore] = (df_cfnai[col] - df_cfnai[col].mean())/df_cfnai[col].std(ddof=0)
        df_cfnai[col_chg] = df_cfnai[col].diff()
        df_cfnai[col_chg_zscore] = (df_cfnai[col_chg] - df_cfnai[col_chg].mean())/df_cfnai[col_chg].std(ddof=0)
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')
        
df_cfnai2 = df_cfnai[col_zscore_list+col_zscore_chg_list]
df_cfnai2.drop(['year_zscore','year_chg_zscore','month_zscore','month_chg_zscore'],axis=1,inplace=True)
df_cfnai2.to_csv('./Output/Monthly_Data/chicago_fed_national_activity_index_output.csv')
df_cfnai2.tail(5)

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,year,month,prod_and_income_zscore,employment_zscore,consumption_and_housing_zscore,sales_orders_inventories_zscore,cfnai_zscore,cfnai_ma3_zscore,diffusion_zscore,prod_and_income_chg_zscore,employment_chg_zscore,consumption_and_housing_chg_zscore,sales_orders_inventories_chg_zscore,cfnai_chg_zscore,cfnai_ma3_chg_zscore,diffusion_chg_zscore
656,2021,11,0.601917,0.693222,-0.078586,0.238230,0.559163,0.646589,1.048463,-0.446871,0.056423,-0.473331,-0.10789,-0.207278,0.399129,1.055420
657,2021,12,-0.341459,0.618880,-1.398525,0.381124,0.009553,0.675961,1.048463,-0.802477,-0.057287,-1.149371,0.10637,-0.454509,0.041722,-0.006557
658,2022,1,0.444687,0.321511,2.173075,0.762176,0.749029,0.646589,0.730839,0.664396,-0.227853,3.109681,0.28492,0.608583,-0.047629,-0.803039
659,2022,2,0.785351,0.767565,-0.311516,-0.142822,0.539178,0.631903,0.677902,0.286565,0.340700,-2.163431,-0.67925,-0.174314,-0.025291,-0.139304
660,2022,3,0.706736,0.395853,-0.311516,0.285861,0.439248,0.837508,1.048463,-0.069040,-0.284708,-0.000103,0.32063,-0.083663,0.309778,0.922673
